In [1]:
import numpy as np
import xarray as xr
import dask

### Load dataset

In [2]:
# Tendencies
#ds_budg = xr.open_zarr('/rigel/ocp/users/jt2796/eccov4r3_datasets/anom/budgH_aveZ/0700m/')

In [2]:
# Integrated+Detrended
ds_budg = xr.open_zarr('/rigel/ocp/users/jt2796/eccov4r3_datasets/anom/budgH_aveZ/intT/dtr/0300m/')

In [3]:
# Add vertical and horizontal component for diffusion and the nonlinear advection term
ds_budg['dif'] = ds_budg.hdif + ds_budg.vdif
ds_budg['adv_Uanom_Tanom'] = ds_budg.hadv_Uanom_Tanom + ds_budg.vadv_Uanom_Tanom

In [4]:
ds_budg

<xarray.Dataset>
Dimensions:           (face: 13, i: 90, j: 90, time: 286)
Coordinates:
  * face              (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i                 (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * j                 (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * time              (time) datetime64[ns] 1992-02-13 1992-03-15 1992-04-14 ...
Data variables:
    frc               (time, face, j, i) float64 dask.array<shape=(286, 13, 90, 90), chunksize=(286, 1, 90, 90)>
    hadv_Uanom_Tanom  (time, face, j, i) float64 dask.array<shape=(286, 13, 90, 90), chunksize=(286, 1, 90, 90)>
    hadv_Uanom_Tclim  (time, face, j, i) float64 dask.array<shape=(286, 13, 90, 90), chunksize=(286, 1, 90, 90)>
    hadv_Uclim_Tanom  (time, face, j, i) float64 dask.array<shape=(286, 13, 90, 90), chunksize=(286, 1, 90, 90)>
    hdif              (time, face, j, i) float64 dask.array<shape=(286, 13, 90, 90), chunksize=(286, 1, 90, 90)>
    res               (tim

### Define function to get ratio of covariance over variance

In [5]:
def get_cov_ratio(x,y):
    y_var = np.var(y[0,0,0],ddof=1)
    cov = np.cov(x[0,0,0], y[0,0,0])[0][1]
    cov_ratio = (cov/y_var)
    return cov_ratio.reshape(1,1,1)

In [6]:
ds = xr.Dataset(data_vars={})

# Define y (total tendency)
y = ds_budg.tnd.isel(time=slice(1, -1)).transpose('face','j','i','time').chunk(chunks=(1,1,1,-1))

for varname in ds_budg.data_vars:
    if (varname == 'tnd' or varname == 'vdif' or varname == 'hdif' \
        or varname == 'hadv_Uanom_Tanom' or varname == 'vadv_Uanom_Tanom'):
        continue
        
    # Define x (Anomaly budget terms)
    x = ds_budg[varname].isel(time=slice(1, -1)).transpose('face','j','i','time').chunk(chunks=(1,1,1,-1))
    
    # Map function across all blocks of a dask array
    cov_ratio = dask.array.map_blocks(get_cov_ratio, x.data, y.data, dtype=np.float64, drop_axis=3)
    
    # Save to dataset
    ds[varname] = xr.DataArray(cov_ratio,coords=[ds_budg.face,ds_budg.j,ds_budg.i],dims=['face','j','i'])

In [7]:
ds

<xarray.Dataset>
Dimensions:           (face: 13, i: 90, j: 90)
Coordinates:
  * face              (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * j                 (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * i                 (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
Data variables:
    frc               (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    hadv_Uanom_Tclim  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    hadv_Uclim_Tanom  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    res               (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    vadv_Uanom_Tclim  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    vadv_Uclim_Tanom  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    dif               (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 1, 1)>
    adv_Uanom_Tano

### Save to netcdf

#### Rechunk and compression

In [8]:
opts = {}
for varname in ds.data_vars:
    ds[varname] = ds[varname].chunk(chunks={'face':13,'j':90,'i':90})
    opts[varname] = {'zlib': True}

In [9]:
ds

<xarray.Dataset>
Dimensions:           (face: 13, i: 90, j: 90)
Coordinates:
  * face              (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * j                 (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * i                 (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
Data variables:
    frc               (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    hadv_Uanom_Tclim  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    hadv_Uclim_Tanom  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    res               (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    vadv_Uanom_Tclim  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    vadv_Uclim_Tanom  (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    dif               (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90

#### Save to file

In [10]:
from dask.diagnostics import ProgressBar

In [12]:
# Integrated
with ProgressBar():
    ds.to_netcdf('/rigel/ocp/users/jt2796/eccov4r3_datasets/anom/budgH_aveZ/intT/corr/0700.nc',
                 format='NETCDF4',encoding=opts)

/rigel/home/jt2796/dask/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


[                                        ] | 0% Completed |  2min 39.2s

/rigel/home/jt2796/miniconda/envs/default/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


[########################################] | 100% Completed | 43min 52.7s


In [11]:
# Integrated+Detrended
with ProgressBar():
    ds.to_netcdf('/rigel/ocp/users/jt2796/eccov4r3_datasets/anom/budgH_aveZ/intT/dtr/corr/0300.nc',
                 format='NETCDF4',encoding=opts)

/rigel/home/jt2796/dask/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


[                                        ] | 0% Completed |  2min 38.2s

/rigel/home/jt2796/miniconda/envs/default/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


[########################################] | 100% Completed | 44min 58.7s


### Check file

In [12]:
# Open dataset from file
ds_corr = xr.open_dataset('/rigel/ocp/users/jt2796/eccov4r3_datasets/anom/budgH_aveZ/intT/dtr/corr/0300.nc')

In [13]:
ds_corr

<xarray.Dataset>
Dimensions:           (face: 13, i: 90, j: 90)
Coordinates:
  * face              (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * j                 (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * i                 (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
Data variables:
    frc               (face, j, i) float64 ...
    hadv_Uanom_Tclim  (face, j, i) float64 ...
    hadv_Uclim_Tanom  (face, j, i) float64 ...
    res               (face, j, i) float64 ...
    vadv_Uanom_Tclim  (face, j, i) float64 ...
    vadv_Uclim_Tanom  (face, j, i) float64 ...
    dif               (face, j, i) float64 ...
    adv_Uanom_Tanom   (face, j, i) float64 ...